In [1]:
#================================= Start of importing required packages and libraries =========================================#
from __future__ import print_function
%matplotlib inline
import numpy as np
import torch
from experiment_federated import *
import random
#================================== End of importing required packages and libraries ==========================================#

[AMP] 当前设备: NVIDIA GeForce RTX 3060 Laptop GPU, 是否启用 AMP: False
[AMP] 当前GPU: NVIDIA GeForce RTX 3060 Laptop GPU, 是否启用 AMP: False


In [2]:
import torch
import random
import torch.nn as nn
import subprocess

# ========== 智能 GPU 检测函数 ==========
def get_free_gpu(threshold_mb=1024):
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_free = [int(x) for x in result.strip().split('\n')]
        for idx, mem in enumerate(memory_free):
            if mem > threshold_mb:
                return idx
    except Exception as e:
        print("无法检测GPU状态，默认使用CPU:", e)
    return None

# ========== 设备选择 ==========
free_gpu = get_free_gpu()
if free_gpu is not None and torch.cuda.is_available():
    DEVICE = torch.device(f'cuda:{free_gpu}')
    GPU_NAME = torch.cuda.get_device_name(free_gpu)
    print(f"✅ 使用空闲GPU: cuda:{free_gpu} - {GPU_NAME}")
else:
    DEVICE = torch.device('cpu')
    GPU_NAME = "CPU"
    print("⚠️ 没有可用GPU，改为使用CPU")

# ========== 环境标识 ==========
IS_SERVER = "A100" in GPU_NAME
IS_LOCAL = "3060" in GPU_NAME or "RTX 3060" in GPU_NAME

# ========== 基本超参数配置 ==========
DATASET_NAME = "PATHMNIST"
MODEL_NAME = "CNNPATH"
DD_TYPE = 'NON_IID'
ALPHA = 1
NUM_PEERS = 100
FRAC_PEERS = 1
SEED = 7
random.seed(SEED)
CRITERION = nn.CrossEntropyLoss()
NUM_CLASSES = 9
CLASS_PER_PEER = 9
SAMPLES_PER_CLASS = 582
RATE_UNBALANCE = 1

LABELS_DICT = {
    'Adipose': 0,
    'Background': 1,
    'Debris': 2,
    'Lymphocytes': 3,
    'Mucus': 4,
    'Smooth_muscle': 5,
    'Normal_colon_mucosa': 6,
    'Cancer_stroma': 7,
    'Adenocarcinoma_epithelium': 8
}
SOURCE_CLASS = 3
TARGET_CLASS = 5

# ========== 环境自适应参数配置 ==========
if IS_SERVER:
    GLOBAL_ROUNDS = 200           # 增加全局轮数，让模型充分学习
    LOCAL_EPOCHS = 3              # 本地训练较充分，降低攻击干扰
    LOCAL_BS = 64                 # 中等 batch size，兼顾稳定与速度
    LOCAL_LR = 0.01              # 降低学习率，增强收敛稳定性
    LOCAL_MOMENTUM = 0.9          # 保持稳定惯性
    TEST_BATCH_SIZE = 1000
    USE_LR_SCHEDULER = True       # 使用学习率调度器控制收敛后期收缩

elif IS_LOCAL:
    GLOBAL_ROUNDS = 5
    LOCAL_EPOCHS = 1
    TEST_BATCH_SIZE = 256
    LOCAL_BS = 32
else:
    GLOBAL_ROUNDS = 50
    LOCAL_EPOCHS = 2
    TEST_BATCH_SIZE = 512
    LOCAL_BS = 32

LOCAL_LR = 0.01
LOCAL_MOMENTUM = 0.9

# ========== 输出当前配置 ==========
print(f"[环境] GPU: {GPU_NAME}")
print(f"[配置] DEVICE = {DEVICE}, GLOBAL_ROUNDS = {GLOBAL_ROUNDS}, LOCAL_EPOCHS = {LOCAL_EPOCHS}, TEST_BATCH_SIZE = {TEST_BATCH_SIZE}")


✅ 使用空闲GPU: cuda:0 - NVIDIA GeForce RTX 3060 Laptop GPU
[环境] GPU: NVIDIA GeForce RTX 3060 Laptop GPU
[配置] DEVICE = cuda:0, GLOBAL_ROUNDS = 5, LOCAL_EPOCHS = 1, TEST_BATCH_SIZE = 256


In [3]:
RULE = 'lfighter_dbo'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)


--> Starting experiment...
Data set: PATHMNIST
Data distribution: NON_IID
Aggregation rule: lfighter_dbo
Attack Type: label_flipping
Attackers Ratio: 40.0 %
Malicious Behavior Rate: 100 %
--> Loading of PATHMNIST dataset
--> Dataset has been loaded!
--> Creating CNNPATH model.....
--> Model has been created!
--> Distributing training data among peers
--> Training data have been distributed among peers
--> Creating peets instances

===>Simulation started...

====>Global model training started...



  0%|          | 0/5 [00:00<?, ?it/s]


 | Global training round : 1/5 |

[0.46372822 0.28177735 0.25449443]

Average test loss: 1.9475, Test accuracy: 2587/7180 (36.03%)

Class      - Accuracy
Adipose    - 97.2
Background - 0.1
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.0
Smooth_muscle - 91.0
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 60.5

 | Global training round : 2/5 |

[0.46372443 0.2817842  0.2544914 ]

Average test loss: 1.4644, Test accuracy: 3661/7180 (50.99%)

Class      - Accuracy
Adipose    - 99.1
Background - 100.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 12.5
Smooth_muscle - 74.7
Normal_colon_mucosa - 4.3
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 71.8

 | Global training round : 3/5 |

[0.463725   0.28178442 0.25449055]

Average test loss: 1.3752, Test accuracy: 3637/7180 (50.65%)

Class      - Accuracy
Adipose    - 98.7
Background - 100.0
Debris     - 4.4
Lymphocytes - 0.0
Mucus      - 21.9
Smooth_muscle - 90.5
Normal_colon_mucosa - 2.8
Cancer_stroma - 0.0
A

In [4]:
# Baseline|: FedAvg-no attacks (FL)
RULE = 'fedavg'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)


--> Starting experiment...
Data set: PATHMNIST
Data distribution: NON_IID
Aggregation rule: fedavg
Attack Type: label_flipping
Attackers Ratio: 0 %
Malicious Behavior Rate: 100 %
--> Loading of PATHMNIST dataset
--> Dataset has been loaded!
--> Creating CNNPATH model.....
--> Model has been created!
--> Distributing training data among peers
--> Training data have been distributed among peers
--> Creating peets instances

===>Simulation started...

====>Global model training started...



  0%|          | 0/5 [00:00<?, ?it/s]


 | Global training round : 1/5 |


Average test loss: 1.8942, Test accuracy: 3020/7180 (42.06%)

Class      - Accuracy
Adipose    - 98.9
Background - 60.4
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.0
Smooth_muscle - 0.7
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 95.8

 | Global training round : 2/5 |


Average test loss: 1.4213, Test accuracy: 3620/7180 (50.42%)

Class      - Accuracy
Adipose    - 98.8
Background - 100.0
Debris     - 24.8
Lymphocytes - 0.2
Mucus      - 19.1
Smooth_muscle - 60.8
Normal_colon_mucosa - 0.5
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 65.2

 | Global training round : 3/5 |


Average test loss: 1.2722, Test accuracy: 4079/7180 (56.81%)

Class      - Accuracy
Adipose    - 98.7
Background - 100.0
Debris     - 3.2
Lymphocytes - 46.2
Mucus      - 22.5
Smooth_muscle - 83.6
Normal_colon_mucosa - 0.9
Cancer_stroma - 16.6
Adenocarcinoma_epithelium - 65.1

 | Global training round : 4/5 |


Average test loss: 1.2015, Test ac

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global training round : 1/5 |


Average test loss: 1.9110, Test accuracy: 3224/7180 (44.90%)

Class      - Accuracy
Adipose    - 97.9
Background - 59.3
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.0
Smooth_muscle - 76.2
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 77.9

 | Global training round : 2/5 |


Average test loss: 1.4721, Test accuracy: 3542/7180 (49.33%)

Class      - Accuracy
Adipose    - 98.8
Background - 100.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 10.6
Smooth_muscle - 94.3
Normal_colon_mucosa - 0.1
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 57.1

 | Global training round : 3/5 |


Average test loss: 1.3763, Test accuracy: 3715/7180 (51.74%)

Class      - Accuracy
Adipose    - 98.5
Background - 100.0
Debris     - 0.9
Lymphocytes - 0.0
Mucus      - 24.9
Smooth_muscle - 96.6
Normal_colon_mucosa - 1.1
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 57.5

 | Global training round : 4/5 |


Average test loss: 1.2899, Test accu

In [5]:
RULE = 'lfighter'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)


--> Starting experiment...
Data set: PATHMNIST
Data distribution: NON_IID
Aggregation rule: lfighter
Attack Type: label_flipping
Attackers Ratio: 40.0 %
Malicious Behavior Rate: 100 %
--> Loading of PATHMNIST dataset
--> Dataset has been loaded!
--> Creating CNNPATH model.....
--> Model has been created!
--> Distributing training data among peers
--> Training data have been distributed among peers
--> Creating peets instances

===>Simulation started...

====>Global model training started...



  0%|          | 0/5 [00:00<?, ?it/s]


 | Global training round : 1/5 |

Potential source and target classes: [1 3]

Average test loss: 2.0383, Test accuracy: 1767/7180 (24.61%)

Class      - Accuracy
Adipose    - 87.8
Background - 0.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.0
Smooth_muscle - 100.0
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 0.0

 | Global training round : 2/5 |

Potential source and target classes: [3 1]

Average test loss: 1.6530, Test accuracy: 3203/7180 (44.61%)

Class      - Accuracy
Adipose    - 99.0
Background - 100.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.4
Smooth_muscle - 100.0
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 35.3

 | Global training round : 3/5 |

Potential source and target classes: [3 1]

Average test loss: 1.5466, Test accuracy: 3372/7180 (46.96%)

Class      - Accuracy
Adipose    - 98.6
Background - 100.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 10.9
Smooth_muscle - 99.5
Normal_colon_mucosa - 0

In [ ]:
RULE = 'median'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)


--> Starting experiment...
Data set: PATHMNIST
Data distribution: NON_IID
Aggregation rule: median
Attack Type: label_flipping
Attackers Ratio: 40.0 %
Malicious Behavior Rate: 100 %
--> Loading of PATHMNIST dataset
--> Dataset has been loaded!
--> Creating CNNPATH model.....
--> Model has been created!
--> Distributing training data among peers
--> Training data have been distributed among peers
--> Creating peets instances

===>Simulation started...

====>Global model training started...



  0%|          | 0/5 [00:00<?, ?it/s]


 | Global training round : 1/5 |


Average test loss: 2.0211, Test accuracy: 1963/7180 (27.34%)

Class      - Accuracy
Adipose    - 96.0
Background - 0.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.0
Smooth_muscle - 100.0
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 7.0

 | Global training round : 2/5 |


Average test loss: 1.5572, Test accuracy: 3431/7180 (47.79%)

Class      - Accuracy
Adipose    - 99.0
Background - 100.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 0.0
Smooth_muscle - 98.6
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 54.7

 | Global training round : 3/5 |


Average test loss: 1.4458, Test accuracy: 3547/7180 (49.40%)

Class      - Accuracy
Adipose    - 98.7
Background - 100.0
Debris     - 0.0
Lymphocytes - 0.0
Mucus      - 9.3
Smooth_muscle - 98.5
Normal_colon_mucosa - 0.0
Cancer_stroma - 0.0
Adenocarcinoma_epithelium - 56.9

 | Global training round : 4/5 |


Average test loss: 1.3533, Test accurac

In [ ]:
RULE = 'tmean'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'mkrum'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'foolsgold'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'Tolpegin'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)

In [ ]:
RULE = 'FLAME'
ATTACK_TYPE='label_flipping'
MALICIOUS_BEHAVIOR_RATE = 1
for atr in [0.4]:
    run_exp(dataset_name = DATASET_NAME, model_name = MODEL_NAME, dd_type = DD_TYPE, num_peers = NUM_PEERS, 
            frac_peers = FRAC_PEERS, seed = SEED, test_batch_size = TEST_BATCH_SIZE,
                criterion = CRITERION, global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS, 
                 local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, labels_dict = LABELS_DICT, device = DEVICE,
                attackers_ratio = atr, attack_type=ATTACK_TYPE, 
                 malicious_behavior_rate = MALICIOUS_BEHAVIOR_RATE, rule = RULE,
                source_class = SOURCE_CLASS, target_class = TARGET_CLASS,
               class_per_peer = CLASS_PER_PEER, samples_per_class = SAMPLES_PER_CLASS, 
               rate_unbalance = RATE_UNBALANCE, alpha = ALPHA, resume = False)